In [1]:
import joblib
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,log_loss
import joblib  # Import joblib for loading the trained model


nltk.data.path.append("/path/to/your/nltk_data")
# Download necessary NLTK data

nltk.download('punkt', download_dir="/path/to/your/nltk_data")

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, Embedding, Masking, Bidirectional, SpatialDropout1D, Flatten
from tensorflow.keras.layers import GlobalMaxPool1D, Flatten, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv(r'C:\Users\sibik\Downloads\SaAndRaDataset.csv')
df                 

,Comment,Sentiment
0,The site has demonstrated a high level of comm...,positive
1,""" The site has implemented comprehensiveitutio...",neutral
2,""" All informed consent forms have been duly si...",positive
3,""" A minor data discrepancy was identifiedPC th...",neutral
4,""" A serious adverse event related to the study...",negative
...,...,...
99995,""" Subject compliance with study medication adm...",negative
99996,""" An equipment malfunction has been reported ,...",negative
99997,##ort site has implemented effective participa...,neutral
99998,##100 The site has consistently maintained pro...,positive


In [4]:
# Define a function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize and remove stopwords
    preprocessed_tokens = []
    stop_words = set(stopwords.words('english'))

    for token in tokens:
        if token not in stop_words:
            preprocessed_tokens.append(lemmatizer.lemmatize(token))

    # Join tokens back into a string
    preprocessed_text = ' '.join(preprocessed_tokens)

    return preprocessed_text

In [5]:
# Apply text preprocessing to the 'Comments' column and store it in a new column
df['preprocessed_text'] = df['Comment'].apply(preprocess_text)
df

,Comment,Sentiment,preprocessed_text
0,The site has demonstrated a high level of comm...,positive,site demonstrated high level commitment engage...
1,""" The site has implemented comprehensiveitutio...",neutral,site implemented comprehensiveitution adverse ...
2,""" All informed consent forms have been duly si...",positive,informed consent form duly signed documented s...
3,""" A minor data discrepancy was identifiedPC th...",neutral,minor data discrepancy identifiedpc source dat...
4,""" A serious adverse event related to the study...",negative,serious adverse event related study drug occur...
...,...,...,...
99995,""" Subject compliance with study medication adm...",negative,subject compliance study medication administra...
99996,""" An equipment malfunction has been reported ,...",negative,equipment malfunction reported ething collecti...
99997,##ort site has implemented effective participa...,neutral,ort site implemented effective participant fol...
99998,##100 The site has consistently maintained pro...,positive,site consistently maintained proper temperatur...


In [6]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df['preprocessed_text'], df['Sentiment'], test_size=0.2, random_state=42)

In [37]:
# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a LinearSVC model
model = LinearSVC()
model.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

C:\Users\sibik\anaconda3\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.9965


In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Train a Multinomial Naive Bayes model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.9848


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust n_estimators as needed
model.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.99625


In [7]:
le = LabelEncoder()
y = le.fit_transform(df["Sentiment"])

In [8]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df["preprocessed_text"].values)
X = tokenizer.texts_to_sequences(df["preprocessed_text"].values)
X = pad_sequences(X)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state = 42, stratify=y)
embed_dim = 128


In [9]:
def create_lstm_model():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(len(le.classes_),activation='softmax'))
    return model

In [10]:
model = create_lstm_model()

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [12]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 22, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 22, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 276,707
Trainable params: 276,707
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(X_train, y_train, epochs = 10, batch_size=32, verbose = 2, validation_split=0.2)

Epoch 1/10
1675/1675 - 84s - loss: 0.0585 - accuracy: 0.9782 - val_loss: 0.0095 - val_accuracy: 0.9948 - 84s/epoch - 50ms/step
Epoch 2/10
1675/1675 - 70s - loss: 0.0097 - accuracy: 0.9948 - val_loss: 0.0088 - val_accuracy: 0.9948 - 70s/epoch - 42ms/step
Epoch 3/10
1675/1675 - 74s - loss: 0.0094 - accuracy: 0.9948 - val_loss: 0.0087 - val_accuracy: 0.9949 - 74s/epoch - 44ms/step
Epoch 4/10
1675/1675 - 70s - loss: 0.0092 - accuracy: 0.9949 - val_loss: 0.0087 - val_accuracy: 0.9948 - 70s/epoch - 42ms/step
Epoch 5/10
1675/1675 - 73s - loss: 0.0083 - accuracy: 0.9952 - val_loss: 0.0086 - val_accuracy: 0.9950 - 73s/epoch - 44ms/step
Epoch 6/10
1675/1675 - 82s - loss: 0.0082 - accuracy: 0.9951 - val_loss: 0.0080 - val_accuracy: 0.9947 - 82s/epoch - 49ms/step
Epoch 7/10
1675/1675 - 77s - loss: 0.0078 - accuracy: 0.9950 - val_loss: 0.0076 - val_accuracy: 0.9949 - 77s/epoch - 46ms/step
Epoch 8/10
1675/1675 - 81s - loss: 0.0079 - accuracy: 0.9950 - val_loss: 0.0076 - val_accuracy: 0.9950 - 81s/ep

In [14]:
y_pred = model.predict(X_test)

# Convert y_pred to a 1D array of class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert y_test to a 1D array of class labels
y_test_labels = np.argmax(y_test, axis=1)

# Calculate the overall accuracy
overall_accuracy = accuracy_score(y_test_labels, y_pred_labels)

# Print the overall accuracy
print('Overall Accuracy:', overall_accuracy)

1032/1032 [==============================] - 12s 9ms/step
Overall Accuracy: 0.9944545454545455


In [18]:
def create_cnn_model():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
    model.add(Conv1D(64, 2, padding='same', activation='relu'))
    model.add(MaxPooling1D(2))  
    model.add(Flatten())
    model.add(Dense(units=1024,activation="relu"))
    model.add(Dense(units=512,activation="relu"))
    model.add(Dense(len(le.classes_),activation='softmax'))
    return model

In [19]:
model = create_cnn_model()
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [20]:
history = model.fit(X_train, y_train, epochs = 10, batch_size=32, verbose = 2, validation_split=0.2)

Epoch 1/10
1675/1675 - 43s - loss: 0.0269 - accuracy: 0.9886 - val_loss: 0.0199 - val_accuracy: 0.9934 - 43s/epoch - 26ms/step
Epoch 2/10
1675/1675 - 46s - loss: 0.0081 - accuracy: 0.9952 - val_loss: 0.0064 - val_accuracy: 0.9954 - 46s/epoch - 28ms/step
Epoch 3/10
1675/1675 - 66s - loss: 0.0112 - accuracy: 0.9951 - val_loss: 0.0102 - val_accuracy: 0.9940 - 66s/epoch - 39ms/step
Epoch 4/10
1675/1675 - 68s - loss: 0.0074 - accuracy: 0.9954 - val_loss: 0.0063 - val_accuracy: 0.9953 - 68s/epoch - 40ms/step
Epoch 5/10
1675/1675 - 95s - loss: 0.0066 - accuracy: 0.9959 - val_loss: 0.0064 - val_accuracy: 0.9954 - 95s/epoch - 57ms/step
Epoch 6/10
1675/1675 - 132s - loss: 0.0063 - accuracy: 0.9957 - val_loss: 0.0067 - val_accuracy: 0.9959 - 132s/epoch - 79ms/step
Epoch 7/10
1675/1675 - 134s - loss: 0.0084 - accuracy: 0.9954 - val_loss: 0.0067 - val_accuracy: 0.9953 - 134s/epoch - 80ms/step
Epoch 8/10
1675/1675 - 119s - loss: 0.0065 - accuracy: 0.9960 - val_loss: 0.0064 - val_accuracy: 0.9952 - 1

In [15]:
import pandas as pd

# Replace 'input.xlsx' with the name of your Excel file
excel_file = pd.read_excel(r'C:\Users\sibik\OneDrive\ドキュメント\eg.xlsx')
# Replace 'output.csv' with the name you want for your CSV file
excel_file.to_csv('eg.csv', index=False)


In [16]:
data = pd.read_csv(r'C:\Users\sibik\Downloads\eg.csv')
data

,Comment
0,"""Serious adverse events have been reported for..."
1,"""Minor deviations from the protocol were noted..."
2,"""All investigational product shipments were re..."
3,"""The site experienced a significant data breac..."
4,"""The site has completed all required documenta..."
5,"""The study team encountered challenges in recr..."
6,"""An unexpected adverse event was reported, and..."
7,"""The trial is progressing well, and interim an..."
8,"""A participant withdrew their consent from the..."
9,"""There was a delay in the delivery of investig..."


In [17]:
model.save("sentence.h5")  # Save as an HDF5 file
from tensorflow.keras.models import load_model
loaded_model = load_model("sentence.h5")


In [18]:
# Define a function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize and remove stopwords
    preprocessed_tokens = []
    stop_words = set(stopwords.words('english'))

    for token in tokens:
        if token not in stop_words:
            preprocessed_tokens.append(lemmatizer.lemmatize(token))

    # Join tokens back into a string
    preprocessed_text = ' '.join(preprocessed_tokens)

    return preprocessed_text

In [19]:
# Apply text preprocessing to the 'Comments' column and store it in a new column
data['preprocessed_text'] = data['Comment'].apply(preprocess_text)
data

,Comment,preprocessed_text
0,"""Serious adverse events have been reported for...",serious adverse event reported two participant...
1,"""Minor deviations from the protocol were noted...",minor deviation protocol noted monitoring visi...
2,"""All investigational product shipments were re...",investigational product shipment received site...
3,"""The site experienced a significant data breac...",site experienced significant data breach compr...
4,"""The site has completed all required documenta...",site completed required documentation subject ...
5,"""The study team encountered challenges in recr...",study team encountered challenge recruiting pa...
6,"""An unexpected adverse event was reported, and...",unexpected adverse event reported investigatio...
7,"""The trial is progressing well, and interim an...",trial progressing well interim analysis show p...
8,"""A participant withdrew their consent from the...",participant withdrew consent trial due persona...
9,"""There was a delay in the delivery of investig...",delay delivery investigational product team ma...


In [20]:
def preprocess_descriptions(descriptions):
    preprocessed_descriptions = [preprocess_text(description) for description in descriptions]
    return preprocessed_descriptions

In [21]:
descriptions = pd.read_csv(r'C:\Users\sibik\Downloads\eg.csv')
descriptions

,Comment
0,"""Serious adverse events have been reported for..."
1,"""Minor deviations from the protocol were noted..."
2,"""All investigational product shipments were re..."
3,"""The site experienced a significant data breac..."
4,"""The site has completed all required documenta..."
5,"""The study team encountered challenges in recr..."
6,"""An unexpected adverse event was reported, and..."
7,"""The trial is progressing well, and interim an..."
8,"""A participant withdrew their consent from the..."
9,"""There was a delay in the delivery of investig..."
